In [1]:
import cv2
from typing import Any
import subprocess
import os
from cv2.typing import MatLike
from argparse import ArgumentParser
import numpy as np
from mirage.mirage_helpers import *
# from pose_extract_base import MLAbstractInterface
# from rgb_interface import CameraInterface
# from movenet import MovenetInterface
# from skeleton import SkeletonDetection


ModuleNotFoundError: No module named 'skeleton'